In [1]:
import pprint
import os
import sys
import numpy as np
from params import get_cfg
from rnn_base import RNN
from ops import Operations
from session import run_session, restore_selection_matrixes
from data_gen import samples_generator, split_train_test
import tensorflow as tf
import pickle

path = '/home/user/neural_program_synthesis/models/summaries/RNN/np_mult-5ops/200_state_size-1500_num_samples-100_batch_size-0.005_learning_rate-1000.0_grad_norm-3_num_features-True_norm/'
#get the global configuration
cfg_path = path+'cfg.p'
cfg = pickle.load(open(cfg_path, 'rb'))
#instantiate containter with the operations avail for the selection
ops = Operations(cfg)
#reset graph so that tf dont complain about not found vars
tf.reset_default_graph()
#generate data 
x,y = samples_generator(cfg['train_fn'], (cfg['num_samples'], cfg['num_features']) , cfg['samples_value_rng'], cfg['seed'])
x_train, x_test, y_train, y_test = split_train_test (x, y , cfg['test_ratio'])
# instanitae the model graph
model = RNN(cfg, ops)
#restore with given path
model_path = path+'/model'
_total_loss_traind_train, _softmaxes_traind_train, _total_loss_traind_test, _softmaxes_traind_test,_total_loss_testd_train, _softmaxes_testd_train, _total_loss_testd_test, _softmaxes_testd_test = restore_selection_matrixes(model, cfg, x_train, x_test, y_train, y_test, model_path)

IndentationError: unexpected indent (session.py, line 270)

In [ ]:
def print_matrix(matrix_lst):
    np.set_printoptions(precision=3, suppress=True)
    for elem in range(len(matrix_lst[0])):
        print("\n##Elem-"+str(elem)+"--#############################################################")        
        for matrix in matrix_lst:
            print(matrix[elem], end=" ")
       

In [ ]:
print_matrix(_softmaxes_train)

In [ ]:
print_matrix(_softmaxes_test)